In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import Dropdown, IntSlider, Button, VBox, HBox, Output
from IPython.display import display, Markdown

from src.recommender import load_shows, Preferences, recommend

shows = load_shows("../data/shows.json")
print("Shows loaded:", len(shows))


Shows loaded: 22


In [3]:
mood_dd = Dropdown(
    options=["comfort","light","thoughtful","dark","intense","romantic","calm","uplifting","awe","nostalgic"],
    value="comfort",
    description="Mood:"
)

genre_dd = Dropdown(
    options=["comedy","drama","thriller","crime","sci_fi","romance","documentary","reality","history"],
    value="comedy",
    description="Genre:"
)

time_sl = IntSlider(value=45, min=15, max=120, step=5, description="Time (min):")
pacing_sl = IntSlider(value=2, min=1, max=4, step=1, description="Pacing:")
intensity_sl = IntSlider(value=2, min=1, max=4, step=1, description="Intensity:")
binge_sl = IntSlider(value=4, min=1, max=5, step=1, description="Binge:")

btn = Button(description="Recommend", button_style="success")
out = Output()

import ipywidgets as widgets

display(VBox([
    widgets.HTML("<h2>Netflix: What Should You Watch?</h2>"),
    HBox([mood_dd, genre_dd]),
    time_sl,
    HBox([pacing_sl, intensity_sl, binge_sl]),
    btn,
    out
]))



In [4]:
def on_click(b):
    with out:
        out.clear_output()
        print("✅ Click registered")

        try:
            pref = Preferences(
                mood=mood_dd.value,
                genre=genre_dd.value,
                time_available=int(time_sl.value),
                pacing=int(pacing_sl.value),
                intensity=int(intensity_sl.value),
                binge=int(binge_sl.value)
            )

            print("Prefs:", pref)

            recs = recommend(shows, pref, top_n=3)
            print("Recs returned:", type(recs), "count:", len(recs) if recs is not None else None)

            if not recs:
                print("⚠️ No recommendations returned. (Empty list)")
                return

            display(Markdown("### Top picks for you ✅"))

            for i, r in enumerate(recs, start=1):
                # show keys so we can't get stuck on a wrong field name
                print(f"\nItem {i} keys:", list(r.keys()))

                title = r.get("title", "Untitled")
                minutes = r.get("episode_minutes", r.get("episode_length", r.get("minutes", "N/A")))
                score = r.get("score", "N/A")

                display(Markdown(
                    f"**{i}. {title}**  \n"
                    f"- Episode length: {minutes}  \n"
                    f"- Score: `{score}`"
                ))

        except Exception as e:
            print("❌ Error inside click handler:")
            print(type(e).__name__, ":", e)

# IMPORTANT: re-bind AFTER defining the function
btn.on_click(on_click)
print("✅ Handler bound to button")


✅ Handler bound to button


In [5]:
display(out)


Output()